In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cmasher as cmr

from fiberamp.fiber.microstruct.bragg import BraggExact
from fiberamp.fiber.microstruct.bragg import plotlogf, plotlogf_real
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton
from ipywidgets import interactive, FloatSlider, Layout

In [ ]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d]

n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air]

mats = ['air', 'glass', 'air']

maxhs = [1, 1, 1]

A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs, no_mesh=True)


In [ ]:
nu = 1

def det_plot(wl):

    A.wavelength = wl * 1e-6
        
    cutoff = A.ks[0] * A.scale
    L, R = .9995, 1.0001
    imin, imax = -.003, .001
    Xr = np.linspace(L * cutoff , R * cutoff , num=75)
    Xi = np.linspace(imin, imax, num=75)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, figsize=(14, 10))
    
    fs1 = A.determinant(zs, nu=nu, outer='h2')
    ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=75)
    ax1.grid(True)
    ax1.set_facecolor('grey')
#     ax1.set_yticks([0])
#     ax1.set_xticks([])

    plt.show()
    
interactive_plot = interactive(det_plot, wl=FloatSlider(min=1.4, max=1.6, step=.01, value=1.5, 
                                             readout_format='.6f', layout=Layout(width='100%')))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

In [ ]:
k_low = A.ks[0] * A.scale
guess = .9999 * k_low
guess

In [ ]:
nu = 1
outer='h2'
beta1 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer)))


In [ ]:
beta1.real/ k_low